In [1]:
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from dataclasses import dataclass

from pydrake.all import (MathematicalProgram, Solve, MonomialBasis,
                         DiagramBuilder, Evaluate, LogVectorOutput, Simulator,
                         SymbolicVectorSystem, Variable, ToLatex, Polynomial,
                         VectorSystem, eq, ge, le, Formula, Expression, Evaluate,
                         LeafSystem, AbstractValue)

from dccm_quasistatic.utils.math_utils import matrix_inverse



In [2]:
def create_numpy_array(string):
    # Split the string by space
    values = string.split()
    # Convert the values to float
    return np.array([float(value) for value in values])

w11c_ans_str = "4.5868 -0.0237 0.1742 2.0684 0.1005 2.7412 -0.0038 -0.0333 0.1304 0.2714 2.4268 -0.2897 2.1171 0.0132 6.9634 0.0005 0.0150 -0.0203 0.0001 0.0031 0.0000 0.0001 0.0006 0.0934 0.0116 0.0234 -0.0000 0.0000"
w12c_ans_str = "-1.8328 0.0654 -0.1007 -0.3102 -0.0460 -2.5568 0.0001 0.0177 -0.0427 -0.0515 -0.2710 0.0327 -0.2791 0.0207 -1.3630 -0.0000 -0.0015 0.0048 -0.0120 0.0013 0.0000 0.0000 -0.0001 -0.0092 -0.0054 -0.0031 -0.0000 0.0000"
w22c_ans_str = "7.2139 -0.0124 0.0012 0.0618 0.0954 1.1859 0.0000 -0.0034 0.0088 0.0296 0.0303 -0.0002 0.0377 0.0987 0.4190 0.0000 0.0001 -0.0010 0.0016 -0.0007 -0.0000 0.0000 0.0000 0.0013 0.0012 0.0059 -0.0000 0.0000"
l1c_ans_str = "-3.3514 -0.0118 0.2920 -2.0838 0.1256 -1.6818 0.0136 -0.2138 0.1707 0.0306 -2.6709 0.3296 -2.2965 0.1873 -4.8506 -0.0282 0.2366 -0.1405 0.5670 -0.1170 0.6971 -0.0001 0.0009 -0.0998 -0.0076 -0.0427 0.0003 0.0000"
l2c_ans_str = "0.1711 0.6323 -0.1381 0.2945 -0.4221 0.3728 0.0011 0.0007 0.0482 -0.3245 0.2982 -0.0407 0.2632 -0.3511 0.4786 0.0031 -0.0261 0.0159 -0.0331 0.0474 -0.1364 0.0000 0.0001 0.0097 0.0053 0.0016 0.0001 0.0000"

w11c_ans = create_numpy_array(w11c_ans_str)
w12c_ans = create_numpy_array(w12c_ans_str)
w22c_ans = create_numpy_array(w22c_ans_str)
l1c_ans = create_numpy_array(l1c_ans_str)
l2c_ans = create_numpy_array(l2c_ans_str)

## Closed form A and B matrices

In [ ]:
prog = MathematicalProgram()
deg = 6
# Convergence rate = 1-beta
beta = 0.1
# Indeterminates
x = prog.NewIndeterminates(2, 'x_{k}')
u = prog.NewIndeterminates(1, 'u_{k}')
w = prog.NewIndeterminates(4, 'w')

# A and B matrices
Ak = np.array([[1.1-0.1*x[1],   0],
               [0.1         ,   0.9]])
Bk = np.array([1, 0])[:, np.newaxis]

# Monomial basis
v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)]
v_next = [vk.Substitute({x[0]: 1.1*x[0] - 0.1*x[0]*x[1] + u[0],
                         x[1]: 0.9*x[1] + 0.9*x[0]}) for vk in v]

# print("v: ", v)
# print("v_next: ", v_next)
dim_v = len(v)
print(f"dim_v: {dim_v}")
w11c = prog.NewContinuousVariables(dim_v, 'w11c')
w12c = prog.NewContinuousVariables(dim_v, 'w12c')
w22c = prog.NewContinuousVariables(dim_v, 'w22c')
# print("w11c: ", w11c)

W11k = w11c.dot(v)
W12k = w12c.dot(v)
W22k = w22c.dot(v)
Wk = np.array([[W11k, W12k], [W12k, W22k]])
# print("W11k: ", W11k)


W11k_next = w11c.dot(v_next)
W12k_next = w12c.dot(v_next)
W22k_next = w22c.dot(v_next)
Wk_next = np.array([[W11k_next, W12k_next], [W12k_next, W22k_next]])

r = prog.NewContinuousVariables(1, 'r')

l1c = prog.NewContinuousVariables(dim_v, 'l1c')
l2c = prog.NewContinuousVariables(dim_v, 'l2c')

L1k = l1c.dot(v)
L2k = l2c.dot(v)
Lk = np.array([[L1k, L2k]])

print("Wk: ", Wk.shape)
print("Ak: ", Ak.shape)
print("Bk: ", Bk.shape)
print("Lk: ", Lk.shape)

prog.AddLinearCost(r[0])
prog.AddLinearConstraint(r[0] >= 0)

v = np.array(v).reshape(-1, 1)
print("v: ", v.shape)
w = np.array(w).reshape(-1, 1)

cross_diag = Ak @ Wk + Bk @ Lk
omega = np.block([[Wk_next, cross_diag],
                 [cross_diag.T, (1-beta)*Wk]])
print("omega: ", omega.shape)
# Note: w is an additional indeterminate that enforces that omega is PSD

prog.AddSosConstraint((w.T @ omega @ w - r[0]).flatten()[0])

# Verify that the solution is meets constraints:
# prog.SetInitialGuess(w11c, w11c_ans)
# prog.SetInitialGuess(w12c, w12c_ans)
# prog.SetInitialGuess(w22c, w22c_ans)
# prog.SetInitialGuess(l1c, l1c_ans)
# prog.SetInitialGuess(l2c, l2c_ans)

result = Solve(prog)
print("Solver succeeded: ", result.is_success())

infeasible_constraints = result.GetInfeasibleConstraints(prog)
for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")

# Extract the solution
print("w11c:\n", result.GetSolution(w11c))
print("\nw12c:\n", result.GetSolution(w12c))
print("\nw22c:\n", result.GetSolution(w22c))
print("\nl1c:\n", result.GetSolution(l1c))
print("\nl2c:\n", result.GetSolution(l2c))

w11c_ans = result.GetSolution(w11c)
w12c_ans = result.GetSolution(w12c)
w22c_ans = result.GetSolution(w22c)
l1c_ans = result.GetSolution(l1c)
l2c_ans = result.GetSolution(l2c)


## Sampled A and B matrices

In [4]:
prog = MathematicalProgram()
deg = 6
# Convergence rate = 1-beta
beta = 0.1
# Indeterminates
x = prog.NewIndeterminates(2, 'x_{k}')
u = prog.NewIndeterminates(1, 'u_{k}')
w = prog.NewIndeterminates(4, 'w')
w = np.array(w).reshape(-1, 1)

# Monomial basis
v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)]
dim_v = len(v)
w11c = prog.NewContinuousVariables(dim_v, 'w11c')
w12c = prog.NewContinuousVariables(dim_v, 'w12c')
w22c = prog.NewContinuousVariables(dim_v, 'w22c')
# print("w11c: ", w11c)

l1c = prog.NewContinuousVariables(dim_v, 'l1c')
l2c = prog.NewContinuousVariables(dim_v, 'l2c')

r = prog.NewContinuousVariables(1, 'r')

lb = -3
ub = 3
N = 20
x_samples = np.random.uniform(lb, ub, (N, 2))
u_samples = np.random.uniform(lb, ub, (N, 1))

for i in range(N):
    xi = x_samples[i, :]
    ui = u_samples[i, :]
    # A and B matrices
    Ak = np.array([[1.1-0.1*xi[1],   0],
                [0.1         ,   0.9]])
    Bk = np.array([1, 0])[:, np.newaxis]

    # Create mapping of variables to values
    env = dict(zip(x, xi))
    # Substitute xi into v(xi)
    v_xi = Evaluate(v, env).flatten()

    xi_next = [1.1*xi[0] - 0.1*xi[0]*xi[1] + ui[0],
               0.9*xi[1] + 0.9*xi[0]]
    # Create mapping of variables to values
    env = dict(zip(x, xi_next))
    # Substitute xi_next into v(xi_next)
    v_xi_next = Evaluate(v, env).flatten()

    W11k = w11c.dot(v_xi)
    W12k = w12c.dot(v_xi)
    W22k = w22c.dot(v_xi)
    Wk = np.array([[W11k, W12k], [W12k, W22k]])
    # print("W11k: ", W11k)

    W11k_next = w11c.dot(v_xi_next)
    W12k_next = w12c.dot(v_xi_next)
    W22k_next = w22c.dot(v_xi_next)
    Wk_next = np.array([[W11k_next, W12k_next], [W12k_next, W22k_next]])


    L1k = l1c.dot(v_xi)
    L2k = l2c.dot(v_xi)
    Lk = np.array([[L1k, L2k]])

    # print("Wk: ", Wk.shape)
    # print("Ak: ", Ak.shape)
    # print("Bk: ", Bk.shape)
    # print("Lk: ", Lk.shape)


    cross_diag = Ak @ Wk + Bk @ Lk
    omega = np.block([[Wk_next, cross_diag],
                    [cross_diag.T, (1-beta)*Wk]])
    # print("omega: ", omega.shape)
    # Note: w is an additional indeterminate that enforces that omega is PSD

    prog.AddSosConstraint((w.T @ omega @ w - r[0] * w.T @ w).flatten()[0])
    

prog.AddLinearCost(r[0])
prog.AddLinearConstraint(r[0] >= 0.01)



# Verify that the solution is meets constraints:
# prog.SetInitialGuess(w11c, w11c_ans)
# prog.SetInitialGuess(w12c, w12c_ans)
# prog.SetInitialGuess(w22c, w22c_ans)
# prog.SetInitialGuess(l1c, l1c_ans)
# prog.SetInitialGuess(l2c, l2c_ans)

result = Solve(prog)
print("Solver succeeded: ", result.is_success())

infeasible_constraints = result.GetInfeasibleConstraints(prog)
for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")

# Extract the solution
print("w11c:\n", result.GetSolution(w11c))
print("\nw12c:\n", result.GetSolution(w12c))
print("\nw22c:\n", result.GetSolution(w22c))
print("\nl1c:\n", result.GetSolution(l1c))
print("\nl2c:\n", result.GetSolution(l2c))
print("r:\n", result.GetSolution(r))

w11c_ans = result.GetSolution(w11c)
w12c_ans = result.GetSolution(w12c)
w22c_ans = result.GetSolution(w22c)
l1c_ans = result.GetSolution(l1c)
l2c_ans = result.GetSolution(l2c)


Solver succeeded:  True
w11c:
 [-1.34316747e-03 -5.72461766e-04  6.24990666e-04  5.29885202e-04
  2.93077195e-03  2.32035047e-04  6.35721762e-05  7.05035161e-03
  4.00257362e-03  2.13378938e-03 -4.45698064e-03 -5.22574344e-04
 -5.03522953e-04  3.43176778e-03 -7.98649857e-04 -3.77025513e-02
  1.31135845e-03 -4.61771443e-03 -7.23057656e-02  2.90633897e-03
  1.66447712e-02  6.98048450e-03  1.13119275e-01 -2.54626103e-02
  4.35648829e-02  2.01815603e-02  2.28855636e-03  5.87382646e-01]

w12c:
 [ 1.07734535e-03 -2.24443736e-04  4.11712370e-03 -5.06736380e-03
  8.35397234e-05 -6.78393902e-03  8.57664951e-04 -6.95172319e-03
 -8.24596877e-03 -8.43176230e-03  1.20446804e-02 -1.88334757e-03
  3.92798934e-03  2.13807816e-03  2.75516159e-02  9.52056606e-03
  5.95502428e-02 -4.87277770e-03  6.28835702e-02  8.24536726e-03
 -1.87360000e-02 -5.91562057e-02 -1.37514892e-01 -1.41649083e-01
  2.47657231e-02  9.89335154e-02  1.14750968e-01 -8.80438288e-02]

w22c:
 [ 1.19983669e-02 -2.66411518e-02  6.96112

# Online Computation

In [ ]:
def display_expression(label:str, expr):
    display(
        Markdown(
            f"${label} = "
            + ToLatex(
                Polynomial(expr)
                .RemoveTermsWithSmallCoefficients(1e-5)
                .ToExpression(),
                6,
            )
            + "$"
        )
    )

In [ ]:
class DesiredTrajectorySource(LeafSystem):
    def __init__(self, dim_x, dim_u):
        LeafSystem.__init__(self)
        self.dim_x = dim_x
        self.dim_u = dim_u
        self.DeclareVectorOutputPort("x_desired", dim_x, self.CalcXDesired)
        self.DeclareVectorOutputPort("u_desired", dim_u, self.CalcUDesired)
    
    def CalcXDesired(self, context, output):
        t = context.get_time()
        if t < 3.3:
            xd = 0
        elif t < 6.6:
            xd = 1
        else:
            xd = 0.5
        # xd = 0
        
        output.SetFromVector(np.ones(self.dim_x) * xd)
    
    def CalcUDesired(self, context, output):
        ud = 0
        output.SetFromVector(np.ones(self.dim_u) * ud)

In [ ]:
@dataclass
class DCCMOnlineParams:
    """
    Parameters for the DCCM controller to run online.
    """
    w11c: np.ndarray
    w12c: np.ndarray
    w22c: np.ndarray
    l1c: np.ndarray
    l2c: np.ndarray

    # System parameters
    dim_x: int = 2 # Dimension of the state
    dim_u: int = 1 # Dimension of the input

    # DCCM Params
    deg: int = 6 # Degree of the polynomial

    # Geodesic calculation parameters
    N: int = 5 # Number of segments to discretize the geodesic into
    
    
class DCCMOnlineController(LeafSystem):
    def __init__(self, params: DCCMOnlineParams):
        LeafSystem.__init__(self)
        self.params = params

        self._geodesic_index = self.DeclareAbstractState(
            AbstractValue.Make(0.0)
        )

        # Inputs
        self._xk_index = self.DeclareVectorInputPort("x_current", self.params.dim_x).get_index()
        self._xd_index = self.DeclareVectorInputPort("x_desired", self.params.dim_x).get_index()
        self._ud_index = self.DeclareVectorInputPort("u_desired", self.params.dim_u).get_index()

        # Outputs
        self.DeclareVectorOutputPort("u_current", self.params.dim_u, self.DoCalcOutput)
        self.DeclareVectorOutputPort("geodesic_current", 1, self.RetrieveGeodesicLength)
    
    def RetrieveGeodesicLength(self, context, output):
        geodesic = context.get_abstract_state(int(self._geodesic_index)).get_value()
        output.SetFromVector([geodesic])
    
    def DoCalcOutput(self, context, output):

        # unpack inputs
        xk = self.get_input_port(self._xk_index).Eval(context)
        xd = self.get_input_port(self._xd_index).Eval(context)
        ud = self.get_input_port(self._ud_index).Eval(context)

        t = context.get_time()
        u, geodesic = self.control_law(xk, xd, ud, t)
        context.get_mutable_abstract_state(int(self._geodesic_index)).set_value(geodesic)
        output.SetFromVector(u)
        
    def control_law(self, xk: np.array, xd: np.array, ud: np.array, t: float = 0) -> np.array:
        # return [1]
        # print(f"t: {t}, xk = {xk}, xd = {xd}, ud = {ud}")
        succeeded, xi, delta_xs, delta_s, geodesic = self.calculate_geodesic(xk, xd)
        if not succeeded:
            print(f"Geodesic calculation failed at time: {t}, u = {ud}")
            return ud, geodesic
    
        x = [Variable(f"x_{i}") for i in range(self.params.dim_x)]
        v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)] # might need to wrap x in Variables()

        # Probably need to set this u* to something else!
        u = ud
        for i in range(self.params.N):
            # Create mapping of variables to values
            env = dict(zip(x, xi[i]))
            # Substitute xi into v(xi)
            v_xi = Evaluate(v, env).flatten()
            # Construct L(xi)
            Li = np.array([[self.params.l1c.dot(v_xi), self.params.l2c.dot(v_xi)]])
            # Construct W(xi)
            W11i = self.params.w11c.dot(v_xi)
            W12i = self.params.w12c.dot(v_xi)
            W22i = self.params.w22c.dot(v_xi)
            Wi = np.array([[W11i, W12i], [W12i, W22i]])
            # Get M(xi) by inverting W(xi)
            Mi = np.linalg.inv(Wi)
            # Add marginal control input to u
            u = u - delta_s[i] * Li @ Mi @ delta_xs[i]
        
        print(f"Geodesic calculation succeeded at time: {t}, u = {u}")

        return u, geodesic

    def calculate_geodesic(self, x0, x1):
        """
        Calculate the geodesic from x0 to x1.
        Based on optimization (27)
        Args:
            x0: (dim_x,): initial state, will correspond to x_k
            x1: (dim_x,): final state, will correspond to x*_k
        """
        prog = MathematicalProgram()
        
        # Numerical state evaluation along the geodesic
        x = prog.NewContinuousVariables(self.params.N + 1, self.params.dim_x, 'x')

        # For optimizing over the epigraph instead of the original objective
        y = prog.NewContinuousVariables(self.params.N, 'y')

        # Displacement vector discretized wrt s parameter
        delta_xs = prog.NewContinuousVariables(self.params.N, self.params.dim_x, '\delta x_s')
        
        # Small positive scaler value
        delta_s = prog.NewContinuousVariables(self.params.N, 's')

        # Add constraint: make sure delta_s's are positive
        si_positive = prog.AddLinearConstraint(ge(delta_s, np.ones_like(delta_s) * 1e-6))

        # Add constraints
        # Constraint 1
        si_sum_to_one = prog.AddLinearConstraint(sum(delta_s) == 1)

        discrete_distances_sum = x0
        # Constraint: Initial state matches x0
        prog.AddConstraint(eq(x[0], x0))
        for i in range(self.params.N):
            discrete_distances_sum = discrete_distances_sum + delta_s[i] * delta_xs[i]
            # Constraint 2: Intermediate state matches sum of deltas

            prog.AddConstraint(eq(x[i+1], discrete_distances_sum))
        # Constraint 3
        total_distances_match = prog.AddConstraint(eq(discrete_distances_sum, x1))
    
        # Sum cost over all segments
        prog.AddCost(np.sum(y))
        # Constraints for the values of y
        for i in range(self.params.N):
            v = [monomial.ToExpression() for monomial in MonomialBasis(x[i], deg)]
            # Construct W(x_i)
            W11i = self.params.w11c.dot(v)
            W12i = self.params.w12c.dot(v)
            W22i = self.params.w22c.dot(v)
            Wi = np.array([[W11i, W12i], [W12i, W22i]])
            # Get M(x_i) by inverting W(x_i)
            Mi = matrix_inverse(Wi) # self.get_2x2_inverse(Wi) # <= because of the division, this is not a polynomial anymore.
            
            # Rational Polynomial Expression
            metric_dist = delta_s[i] * delta_xs[i].T @ Mi @ delta_xs[i]
            # print(f"metric_dist: {metric_dist}")
            # print(f"metric_dist.is_polynomial(): {metric_dist.is_polynomial()}")
            # print(f"metric_dist type: {type(metric_dist)}")
            prog.AddConstraint(metric_dist <= y[i])
        
        # Try to keep delta_s small
        prog.AddCost(np.sum(delta_s**2))

        # Seed initial guess as all 1's so that determinant will not be 0 and cause a failure
        prog.SetInitialGuessForAllVariables(np.ones(prog.num_vars()))

        result = Solve(prog)
        geodesic_length = np.sum(result.GetSolution(y))
        return result.is_success(), result.GetSolution(x), result.GetSolution(delta_xs), result.GetSolution(delta_s), geodesic_length
    
    def get_2x2_inverse(self, A: np.array) -> np.array:
        # This doesn't work: np.linalg.inv(A)
        det = A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]
        return np.array([[A[1, 1], -A[0, 1]], [-A[1, 0], A[0, 0]]]) / det
         
def test_control_law():
    # Test the control law
    x0 = np.array([1, 0.5])
    x1 = np.array([0, 0])
    ud = np.array([0])
    params = DCCMOnlineParams(w11c_ans, w12c_ans, w22c_ans, l1c_ans, l2c_ans,
                              N=10)
    controller = DCCMOnlineController(params)
    res = controller.control_law(x0, x1, ud)
    print("res: ", res)

def test_geodesic():
    # Test the geodesic calculation
    x0 = np.array([1, 0.5])
    x1 = np.array([0, 0])
    ud = np.array([0])
    params = DCCMOnlineParams(w11c_ans, w12c_ans, w22c_ans, l1c_ans, l2c_ans,
                              N=15)
    controller = DCCMOnlineController(params)
    succeeded, x, delta_xs, delta_s, geodesic = controller.calculate_geodesic(x0, x1)
    print("succeeded: ", succeeded)
    print("x: ", x)
    print("delta_xs: ", delta_xs)
    print("delta_s: ", delta_s)

    visualize_geodesic_path(x0, x1, x, delta_xs, delta_s)

def visualize_geodesic_path(x0, x1, x, delta_xs, delta_s):
    """
    Visualize the geodesic path from x0 to x1.
    """

    plt.plot(x[:, 0], x[:, 1], 'o-')
    plt.plot(x0[0], x0[1], 'o', color='red')
    plt.plot(x1[0], x1[1], 'o', color='green')
    plt.show()

# test_geodesic()
# test_control_law()

In [ ]:
params = DCCMOnlineParams(w11c_ans, w12c_ans, w22c_ans, l1c_ans, l2c_ans,
                          deg=deg, N=3)
x1 = Variable('x_{1_k}')
x2 = Variable('x_{2_k}')
u = np.array([Variable('u_k')])
x = np.array([x1, x2])

# A and B matrices
Ak = np.array([[1.1-0.1*x[1],   0],
               [0.1         ,   0.9]])
Bk = np.array([1, 0])[:, np.newaxis]


dynamics= Ak @ x + Bk @ u
display_expression('x_{1_{k+1}}', dynamics[0])
display_expression('x_{2_{k+1}}', dynamics[1])

cstr_sys = SymbolicVectorSystem(state=x,
                           dynamics=dynamics,
                           input=u,
                           output=x,
                           time_period=0.1 # Discrete time system
                           )

builder = DiagramBuilder()
system = builder.AddSystem(cstr_sys)

# DCCM Controller
controller = builder.AddSystem(DCCMOnlineController(params))

# Desired trajectory system
desired_trajectory = builder.AddSystem(DesiredTrajectorySource(params.dim_x, params.dim_u))

builder.Connect(desired_trajectory.GetOutputPort("x_desired"), controller.GetInputPort("x_desired"))
builder.Connect(desired_trajectory.GetOutputPort("u_desired"), controller.GetInputPort("u_desired"))
builder.Connect(system.get_output_port(0), controller.GetInputPort("x_current"))
builder.Connect(controller.GetOutputPort("u_current"), system.get_input_port(0))
# builder.ExportInput(system.get_input_port(0))

# Loggers
x_logger = LogVectorOutput(system.get_output_port(0), builder)
xd_logger = LogVectorOutput(desired_trajectory.GetOutputPort("x_desired"), builder)
u_logger = LogVectorOutput(controller.GetOutputPort("u_current"), builder)
g_logger = LogVectorOutput(controller.GetOutputPort("geodesic_current"), builder)

diagram = builder.Build()

context = diagram.CreateDefaultContext()
context.SetDiscreteState([1, 0.5])

# system_context = diagram.GetMutableSubsystemContext(system, context)
# system.get_input_port(0).FixValue(system_context, [0])

simulator = Simulator(diagram, context)
simulator.AdvanceTo(10)

# Plot the results.
x_log = x_logger.FindLog(context)
xd_log = xd_logger.FindLog(context)
u_log = u_logger.FindLog(context)
g_log = g_logger.FindLog(context)

fig, axs = plt.subplots(3, 1, figsize=(16, 16))

axis = axs[0]
axis.step(x_log.sample_times(),
         x_log.data().transpose(),
         x_log.sample_times(), xd_log.data().transpose()[:,0])
axis.legend([r'$x_{1_k}$', r'$x_{2_k}$', r'$x*$'])
axis.set_ylabel('state')
axis.set_xlabel('t')

axis = axs[1]
axis.step(u_log.sample_times(),
         u_log.data().transpose())
axis.legend([r'$u_{k}$'])
axis.set_ylabel('u')
axis.set_xlabel('t')

axis = axs[2]
axis.step(g_log.sample_times(),
         g_log.data().transpose())
axis.set_ylabel('geodesic length')
axis.set_xlabel('t')

plt.show()